Importing Libraries

In [27]:
import numpy as np
import pandas as pd
import xlsxwriter

wb=xlsxwriter.Workbook('MSE_vs_N.xlsx')
sheet=wb.add_worksheet('Sheet 1')

Reading User Inputs

In [28]:
file=open("Output_Soft_Computing.txt","w+")

l=int(input('Enter number of input neurons (For my dataset it is 6): '))
n=int(input('Enter number of output neurons (For my dataset it is 1): '))
P=int(input('Total number of patterns in my dataset is 1000. Enter the number of training patterns accordingly: '))
no_test=int(input('Enter number of testing patterns: '))
m=8 #number of hidden neurons

file.write("Number of input neurons: ")
file.write(str(l))
file.write("\n\nNumber of hidden neurons: ")
file.write(str(m))
file.write("\n\nNumber of output neurons: ")
file.write(str(n))
file.write("\n\nNumber of training patterns: ")
file.write(str(P))
file.write("\n\nNumber of testing patterns: ")
file.write(str(no_test))
print('Entered number of training patterns is: ',P)
print('Entered number of testing patterns is: ',no_test)

Entered number of training patterns is:  750
Entered number of testing patterns is:  250


Reading Input File

In [29]:
input_file=pd.read_csv('Nozzle_Velocity_Prediction.csv')

Converting Input Dataframe to Array

In [30]:
iparray=input_file.to_numpy()
Iin=iparray[:P+no_test,0:l]
Iin=Iin.transpose()

Normalization of Input Data

In [31]:
I=np.zeros((l,P+no_test))
for idx in range(0,6):
    Imax=max(Iin[idx,])
    Imin=min(Iin[idx,])
    I[idx,]=0.1+(0.8*(Iin[idx,]-Imin)/(Imax-Imin))

Splitting of Input Data into Training and Testing Inputs

In [32]:
X=I[0:l,0:P]
Xtest=I[0:l,P:P+no_test]

Target Normalisation and Splitting of Target Values into Training and Testing Targets

In [33]:
Tin=iparray[:P+no_test,l][np.newaxis]
Tmax=np.max(Tin)
Tmin=np.min(Tin)

Tnorm=0.1+(0.8*((Tin-Tmin)/(Tmax-Tmin)))
T=Tnorm[0,:P][np.newaxis]

Ttest=Tnorm[0,P:P+no_test][np.newaxis]

Generating Random Weights

In [34]:
V=np.random.rand(l,m)
VT=V.transpose()

W=np.random.rand(m,n)
WT=W.transpose()

file.write("\n\nInitial random weights V between Input layer and Hidden layer: \n\n ")
content1=str(V)
file.write(content1)
file.write("\n\nInitial random weights W between Hidden layer and Output layer: \n\n  ")
content2=str(W)
file.write(content2)


112

Generating Random Biases

In [35]:
b1=np.random.rand(m,1)
b2=np.random.rand(n,1)

content3=str(b1)
file.write("\n\nInitial random bias to the hidden layer:\n\n")
file.write(content3)
content4=str(b2)
file.write("\n\nInitial random bias to the output layer:\n\n")
file.write(content4)

14

Coefficients in Log Sigmoid TF, Learning Rate and Momentum Constant

In [36]:
a1=1
a2=1
eta=0.6
alpha=0.5

Feed Forward Calculation

In [37]:
Ih=np.matmul(VT,X)+b1
Oh=np.reciprocal(np.add(1,np.exp(-a1*Ih)))
Io=np.matmul(WT,Oh)+b2
Oo=np.reciprocal(np.add(1,np.exp(-a1*Io)))

Ek=0.5*pow((T-Oo),2)

Esum=0.0
Etemp=np.zeros((n,1))
Ekavg=0

for k in range(0,n):
    for p in range(0,P):
        Esum=(Esum+(Ek[k,p]))
    Etemp[k,0]=Esum/P
Ekavg=0.0
for k in range(0,n):
    Ekavg=Ekavg+(Etemp[k,0]/n)

MSE=2*Ekavg #Mean Square Error

Back Propagation and Updating of Network Parameters

In [38]:
count=0
sheet.write(0,0,'Iteration')
sheet.write(0,1,'MSE')

prevdelW=np.zeros((m,n)) #For momentum term for W
prevdelV=np.zeros((l,m)) #For momentum term for V


while MSE>0.0015:

    delwjk=np.zeros((m,n))
    for j in range(0,m): 
        for k in range(0,n):
            for p in range(0,P):
                delwjk[j,k]=delwjk[j,k]+((T[k,p]-Oo[k,p])*a2*Oo[k,p]*(1-Oo[k,p])*Oh[j,p])*(eta/P)

    W=W+delwjk+(alpha*prevdelW)      #Updating Weigths between input and hidden layer i.e. W matrix
    prevdelW=delwjk

    delvij=np.zeros((l,m))
    for i in range(0,l):
        for j in range(0,m):
            for k in range(0,n):
               for p in range(0,P):
                   delvij[i,j]=delvij[i,j]+((T[k,p]-Oo[k,p])*a2*Oo[k,p]*(1-Oo[k,p])*a1*Oh[j,p]*(1-Oh[j,p])*W[j,k]*X[i,p])*(eta/(n*P))
 
    V=V+delvij+(alpha*prevdelV)     #Updating Weights betweeb hidden and output layer i.e. V matrix
    prevdelV=delvij
    
    delb2=np.zeros((n,1))
    for k in range(0,n):
            for p in range(0,P):
                delb2[k,0]=(delb2[k,0]+(T[k,0]-Oo[k,p])*a2*Oo[k,p]*(1-Oo[k,p]))*(eta/P)
    
    b2=b2+delb2     #Updating bias matrix of output layer

    delb1=np.zeros((m,1))
    for j in range(0,m):
            for k in range(0,n):
               for p in range(0,P):
                   delb1[j,0]+=((T[k,p]-Oo[k,p])*a2*Oo[k,p]*(1-Oo[k,p])*a1*Oh[j,p]*(1-Oh[j,p])*W[j,k])*(eta/(n*P))
    
    b1=b1+delb1 #Updating bias matrix of input layer


    ################################ Feed Forward Calculation after Updating Network Parameters###############################
   
    VT=V.transpose()
    WT=W.transpose()
    Ih=np.matmul(VT,X)+b1
    Oh=np.reciprocal(np.add(1,np.exp(-a1*Ih)))
    Io=np.matmul(WT,Oh)+b2
    Oo=np.reciprocal(np.add(1,np.exp(-a1*Io)))

    
    Ek=0.5*pow((T-Oo),2)
    Esum=0.0
    Etemp=np.zeros((n,1))

    for k in range(0,n):
        for p in range(0,P):
            Esum=(Esum+(Ek[k,p]))
        Etemp[k,0]=Esum/P #avg error of kth neuron for all p patterns
    Ekavg=0.0
    for k in range(0,n):
        Ekavg=Ekavg+(Etemp[k,0]/n) #avg error of all k neurons

    MSE=2*Ekavg #Mean Square Error

    count=count+1
    sheet.write(count,0,count)
    sheet.write(count,1,MSE)

    print("Iteration Number: ",count)
    print('MSE: ',MSE)


wb.close()
content5=str(V)
content6=str(W)
content7=str(b1)
content8=str(b2)
file.write("\n\nFinal weights V aftr training the ANN: \n\n")
file.write(content5)
file.write("\n\nFinal weights W after training the ANN: \n\n")
file.write(content6)
file.write("\n\nFinal bias to the hidden layer after training ANN: \n\n")
file.write(content7)
file.write("\n\nFinal bias to the output layer after training ANN: \n\n")
file.write(content8)


Iteration Number:  1
MSE:  0.42142675704325233
Iteration Number:  2
MSE:  0.40844227319454013
Iteration Number:  3
MSE:  0.39145396121796316
Iteration Number:  4
MSE:  0.36850803545226546
Iteration Number:  5
MSE:  0.3369812578604717
Iteration Number:  6
MSE:  0.29353349719889515
Iteration Number:  7
MSE:  0.23558455364331732
Iteration Number:  8
MSE:  0.16613316837503528
Iteration Number:  9
MSE:  0.09980372504222912
Iteration Number:  10
MSE:  0.05501955383537831
Iteration Number:  11
MSE:  0.033952544723405614
Iteration Number:  12
MSE:  0.026086779359711328
Iteration Number:  13
MSE:  0.023292146865450898
Iteration Number:  14
MSE:  0.022229581166630367
Iteration Number:  15
MSE:  0.021785655786244727
Iteration Number:  16
MSE:  0.02158521726627422
Iteration Number:  17
MSE:  0.021489143266518664
Iteration Number:  18
MSE:  0.021440535635941782
Iteration Number:  19
MSE:  0.021414371204560648
Iteration Number:  20
MSE:  0.021399092570828445
Iteration Number:  21
MSE:  0.02138918418

Testing of the ANN

In [39]:

Ih=np.matmul(VT,Xtest)+b1
Oh=np.reciprocal(np.add(1,np.exp(-a1*Ih)))
Io=np.matmul(WT,Oh)+b2
Oo=np.reciprocal(np.add(1,np.exp(-a1*Io)))


Fin=np.zeros((n,no_test))
Fin=Oo
F=(((Fin-0.1)/0.8)*(Tmax-Tmin))+Tmin
FT=F.transpose()

content9=str(FT)
file.write("\n\nOutput of the Network: \n\n")
file.write(content9)

Ek=0.5*pow((Ttest-Oo),2)
Esum=0.0
Etemp=np.zeros((n,1))
Ekavg=0

for k in range(0,n):
    for p in range(0,no_test):
        Esum=(Esum+(Ek[k,p]))
    Etemp[k,0]=Esum/no_test
#print(Etemp)
Ekavg=0.0
for k in range(0,n):
    Ekavg=Ekavg+(Etemp[k,0]/n)

MSE=2*Ekavg
print(MSE)

file.write("\n\nMean Square Error for Testing Patterns: ")
file.write(MSE)
file.write("\n\nTotal Number of Iterations taken to train the network: ")
file.write(count)
file.close()


0.0018452680925847107
